In [1]:
from pathlib import Path
import random, json
import numpy as np
import torch, os
import torch.nn as nn
from torch.utils.data import DataLoader, Subset, WeightedRandomSampler
from torchvision import transforms, datasets
from PIL import Image
import timm
from tqdm import tqdm
from typing import List, Tuple, Dict, Optional
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve, confusion_matrix, classification_report

try:
    from pytorch_grad_cam import GradCAM
    from pytorch_grad_cam.utils.image import show_cam_on_image
    _HAS_GRADCAM = True
except Exception:
    _HAS_GRADCAM = False

c:\Users\Jomar\VSCode\ML\BasicML\ML\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Create Config metrics for easier variable changing later down the line

In [2]:
DATA_ROOT = Path("/path/to/data")   # <-- set path
TRAIN_DIR, TEST_DIR = DATA_ROOT/"train", DATA_ROOT/"test"
MODEL_DIR = Path("models_ultrasound"); MODEL_DIR.mkdir(parents=True, exist_ok=True)

BACKBONE = "efficientnet_b0"
IMG_SIZE, BATCH_SIZE = 224, 12
NUM_EPOCHS, FREEZE_EPOCHS = 30, 5
LR_HEAD, LR_BACKBONE, WEIGHT_DECAY = 1e-3, 1e-4, 1e-5
PATIENCE_ES, VAL_SPLIT, RANDOM_SEED = 6, 0.15, 42
POS_LABEL_IDX, NUM_WORKERS = 1, 2
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
USE_AMP = True

In [3]:
#for reproduction purposes set all variable random seeds
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

set_seed()


In [5]:
#since expected input are grayscale (ultrasounds) reproduce it into RGB channels so it can work on pretrained models
class Ensure3Channel:
    def __call__(self, img: Image.Image) -> Image.Image:
        return img.convert("RGB")